## Question 2 (Kaggle Competition): 
The goal of this question is to check your knowledge of existing algorithms, your ability to come up with new algorithms and efficiency of your code. Attempt the following kaggle competition and document the results:  https://www.kaggle.com/c/afsis-soil-properties 


In [122]:
# import the necessary libraries
import numpy as np
import pandas as pd
from sklearn import svm, cross_validation

In [123]:
# load the training data
train= pd.read_csv("training.csv")

In [124]:
# let's check it
train.head()

,PIDN,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,...,REF7,RELI,TMAP,TMFI,Depth,Ca,P,pH,SOC,Sand
0,XNhoFZW5,0.302553,0.301137,0.299748,0.300354,0.302679,0.303799,0.301702,0.298936,0.298126,...,-0.646673,1.687734,0.190708,0.056843,Topsoil,-0.295749,-0.041336,-1.129366,0.353258,1.269748
1,9XNspFTd,0.270192,0.268555,0.266964,0.267938,0.271013,0.272346,0.269870,0.266976,0.266544,...,-0.646673,1.687734,0.190708,0.056843,Subsoil,-0.387442,-0.231552,-1.531538,-0.264023,1.692209
2,WDId41qG,0.317433,0.316265,0.314948,0.315224,0.316942,0.317764,0.316067,0.313874,0.313301,...,-0.814516,1.806660,0.190708,0.056843,Topsoil,-0.248601,-0.224635,-0.259551,0.064152,2.091835
3,JrrJf1mN,0.261116,0.259767,0.258384,0.259001,0.261310,0.262417,0.260534,0.258039,0.257246,...,-0.814516,1.806660,0.190708,0.056843,Subsoil,-0.332195,-0.318014,-0.577548,-0.318719,2.118477
4,ZoIitegA,0.260038,0.258425,0.256544,0.257030,0.259602,0.260786,0.258717,0.256352,0.255902,...,-0.780242,0.430513,0.190708,0.056843,Topsoil,-0.438350,-0.010210,-0.699135,-0.310905,2.164148


In [46]:
train.shape

(1157, 3600)

In [125]:
# load testing data
test = pd.read_csv('sorted_test.csv')

In [48]:
test.head()

,PIDN,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,...,LSTD,LSTN,REF1,REF2,REF3,REF7,RELI,TMAP,TMFI,Depth
0,09gt9UK5,0.210899,0.209221,0.207645,0.208714,0.211972,0.213562,0.211267,0.208248,0.207341,...,-0.009394,0.197885,-0.622545,-0.548238,-0.637681,-0.413306,-0.130139,0.154013,-0.033905,Subsoil
1,0BVvxJ6a,0.191650,0.184120,0.177070,0.180624,0.192343,0.197502,0.187655,0.175447,0.172639,...,-0.277944,-0.851643,-0.792000,-1.058442,-0.792961,-0.785786,-0.588855,0.106200,0.559694,Subsoil
2,0KbdgApg,0.083966,0.074740,0.065596,0.069924,0.085145,0.092527,0.080625,0.064768,0.060597,...,-0.443809,-0.628839,-0.754909,-0.926716,-0.865424,-0.607359,-0.334013,-0.021042,0.182431,Topsoil
3,0MnuuduB,0.345962,0.338567,0.331786,0.336948,0.350820,0.356365,0.345285,0.332233,0.328589,...,-0.758853,-0.961080,-0.729455,-0.780148,-0.716356,-0.552923,-0.079171,0.394936,0.774477,Topsoil
4,0PcIsF9z,0.239019,0.232611,0.226266,0.229921,0.241821,0.247756,0.239242,0.227977,0.224762,...,-1.228289,1.156914,-0.696727,0.080705,0.118012,-0.920363,-0.266055,1.612625,1.199048,Subsoil


In [126]:
# segregate the target variables from training data
labels = train[['Ca','P','pH','SOC','Sand']].values

In [50]:
test.describe()

,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,m7480.61,...,EVI,LSTD,LSTN,REF1,REF2,REF3,REF7,RELI,TMAP,TMFI
count,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,...,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000,727.000000
mean,0.232165,0.226979,0.222080,0.225511,0.234976,0.238725,0.231077,0.222129,0.219685,0.219343,...,0.583861,-0.364974,0.269816,-0.668341,-0.572728,-0.594943,-0.640463,0.084040,0.545331,0.703120
std,0.098487,0.099179,0.099853,0.099608,0.098558,0.097966,0.098652,0.099714,0.099947,0.099900,...,0.727844,0.835115,0.883666,0.320982,0.350492,0.410649,0.364303,0.941628,0.682021,0.787634
min,-0.037277,-0.043984,-0.050737,-0.046731,-0.034096,-0.028093,-0.037383,-0.049268,-0.052309,-0.052265,...,-0.800292,-2.366282,-1.760197,-1.161455,-1.360853,-1.267081,-1.268649,-0.605844,-0.541529,-0.772670
25%,0.170926,0.165406,0.160249,0.163639,0.173564,0.177973,0.169524,0.160199,0.157818,0.157650,...,0.164723,-1.046907,-0.546596,-0.885818,-0.792208,-0.910973,-0.929435,-0.401971,-0.021042,0.182431
50%,0.246999,0.242504,0.239130,0.241215,0.249651,0.253969,0.245869,0.238573,0.236578,0.236108,...,0.507289,-0.390770,0.497065,-0.746909,-0.614100,-0.672878,-0.684476,-0.266055,0.501829,0.774477
75%,0.294928,0.290607,0.287255,0.290548,0.299234,0.301754,0.294658,0.286204,0.284108,0.283883,...,1.043003,0.261494,1.042551,-0.547273,-0.439703,-0.378882,-0.412550,0.124703,0.981483,1.053572
max,0.490711,0.486155,0.482087,0.485504,0.494250,0.497684,0.490965,0.483429,0.481667,0.481884,...,2.201895,1.666615,1.844185,0.572364,0.530612,0.681159,0.579637,5.731226,2.180246,2.622005


In [127]:
# We will be using 3 regressors here
columns=['Predicted_svr', 'Predicted_ridge', 'Predicted_regressor']

#### SVR

In [129]:

train.drop(['Ca', 'P', 'pH', 'SOC', 'Sand', 'PIDN'], axis=1, inplace=True)
test.drop('PIDN', axis=1, inplace=True)

xtrain, xtest = np.array(train)[:,:3578], np.array(test)[:,:3578]

sup_vec = svm.SVR(C=10000.0, verbose = 2)

preds1 = np.zeros((xtest.shape[0], 5))
for i in range(5):
    sup_vec.fit(xtrain, labels[:,i])
    preds1[:,i] = sup_vec.predict(xtest).astype(float)

sample = pd.read_csv('sample_submission.csv')
sample['Ca'] = preds1[:,0]
sample['P'] = preds1[:,1]
sample['pH'] = preds1[:,2]
sample['SOC'] = preds1[:,3]
sample['Sand'] = preds1[:,4]

sample.to_csv('svr_predictions.csv', index = False)
#score = 0.462

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [76]:
# create dataframes for all the target variables
# these dataframes will contain prediction values for the respective target variable
df_ca = pd.DataFrame(columns=columns)
df_p = pd.DataFrame(columns=columns)
df_ph = pd.DataFrame(columns=columns)
df_soc = pd.DataFrame(columns=columns)
df_sand = pd.DataFrame(columns=columns)

In [78]:
# storing results for svr 
x = np.zeros((xtrain.shape[0], 5))
for i in range(5):
    x[:,i] = sup_vec.predict(xtrain).astype(float)

df_ca['Predicted_svr'] = x[:,0]
df_p['Predicted_svr'] = x[:,1]
df_ph['Predicted_svr'] = x[:,2]
df_soc['Predicted_svr'] = x[:,3]
df_sand['Predicted_svr'] = x[:,4]

#### Linear Regression

In [130]:
from sklearn import linear_model  
import math
# from sklearn.feature_selection import RFE
# alpha = 8.64304235*math.pow(10,-7)
ridge = linear_model.Ridge(alpha=0.5)  
# selector = RFE(model, 5000, step=1)
# selector = selector.fit(X, y)

preds2 = np.zeros((xtest.shape[0], 5))
for i in range(5):
    ridge.fit(xtrain, labels[:,i])
#     selector = selector.fit(xtrain, labels[:,i])
    preds2[:, i] = ridge.predict(xtest).astype(float)


sample = pd.read_csv('sample_submission.csv')
sample['Ca'] = preds2[:,0]
sample['P'] = preds2[:,1]
sample['pH'] = preds2[:,2]
sample['SOC'] = preds2[:,3]
sample['Sand'] = preds2[:,4]

sample.to_csv('linear_regression_predictions.csv', index = False) 

# score = 0.96048 -- Linear regression, no a
# score = 0.50207 -- Ridge regression, a=.5
# score = 0.51089 -- Lasso regression, a=.1

In [85]:
y = np.zeros((xtrain.shape[0], 5))
for i in range(5):
    y[:,i] = ridge.predict(xtrain).astype(float)


df_ca['Predicted_ridge'] = y[:,0]
df_p['Predicted_ridge'] = y[:,1]
df_ph['Predicted_ridge'] = y[:,2]
df_soc['Predicted_ridge'] = y[:,3]
df_sand['Predicted_ridge'] = y[:,4]

In [ ]:
# # Import the model we are using
# from sklearn.ensemble import RandomForestRegressor
# # Instantiate model with 1000 decision trees
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# # Train the model on training data
# rf.fit(xtrain, train_labels)
# from sklearn.tree import DecisionTreeRegressor
# regressor = DecisionTreeRegressor(random_state=0)

# preds = np.zeros((xtest.shape[0], 5))
# for i in range(5):
#     regressor.fit(xtrain, labels[:,i])
# #     selector = selector.fit(xtrain, labels[:,i])
#     preds[:, i] = regressor.predict(xtest).astype(float)


# sample = pd.read_csv('sample_submission.csv')
# sample['Ca'] = preds[:,0]
# sample['P'] = preds[:,1]
# sample['pH'] = preds[:,2]
# sample['SOC'] = preds[:,3]
# sample['Sand'] = preds[:,4]

# sample.to_csv('beating_benchmark_linear_regression.csv', index = False) 


#### Multi-layer perceptron

In [131]:
from sklearn.neural_network import MLPRegressor
regressor = MLPRegressor(batch_size=100, max_iter=500, solver='lbfgs')

preds3 = np.zeros((xtest.shape[0], 5))
for i in range(5):
    regressor.fit(xtrain, labels[:,i])
#     selector = selector.fit(xtrain, labels[:,i])
    preds3[:, i] = regressor.predict(xtest).astype(float)


sample = pd.read_csv('sample_submission.csv')
sample['Ca'] = preds3[:,0]
sample['P'] = preds3[:,1]
sample['pH'] = preds3[:,2]
sample['SOC'] = preds3[:,3]
sample['Sand'] = preds3[:,4]

sample.to_csv('predictions_mlp_regressor.csv', index = False) 

#score = 0.49056

In [87]:
z = np.zeros((xtrain.shape[0], 5))
for i in range(5):
    z[:,i] = regressor.predict(xtrain).astype(float)

df_ca['Predicted_regressor'] = z[:,0]
df_p['Predicted_regressor'] = z[:,1]
df_ph['Predicted_regressor'] = z[:,2]
df_soc['Predicted_regressor'] = z[:,3]
df_sand['Predicted_regressor'] = z[:,4]

### Averaging results of all the regressors

In [121]:
### Voting Classifier
# from sklearn.ensemble import VotingClassifier
# vc = VotingClassifier(estimators=[('ridge',ridge), ('mlp_regressor',regressor), ('sup_vec',sup_vec)], voting='hard')
# eclf1 = eclf1.fit(X, y)
from sklearn import linear_model
stacker = linear_model.LinearRegression()
preds = np.zeros((xtest.shape[0], 5))

# for Ca
preds[:,0] = [(i+j+k)/3 for i,j,k in zip(preds1[:,0],preds2[:,0],preds3[:,0])]
# stacker.fit(df_ca[['Predicted_svr', 'Predicted_ridge', 'Predicted_regressor']], labels[:,0])
# preds[:, 0] = stacker.predict(np.stack([preds3[:,0], preds2[:,0], preds1[:,0]], axis=-1)).astype(float)

# for p
# stacker.fit(df_p[['Predicted_svr', 'Predicted_ridge', 'Predicted_regressor']], labels[:,1])
# preds[:, 1] = stacker.predict(np.stack([preds3[:,1], preds2[:,1], preds1[:,1]], axis=-1)).astype(float)
preds[:,1] = [(i+j+k)/3 for i,j,k in zip(preds1[:,1],preds2[:,1],preds3[:,1])]

# for ph
# stacker.fit(df_ph[['Predicted_svr', 'Predicted_ridge', 'Predicted_regressor']], labels[:,2])
# preds[:, 2] = stacker.predict(np.stack([preds3[:,2], preds2[:,2], preds1[:,2]], axis=-1)).astype(float)
preds[:,2] = [(i+j+k)/3 for i,j,k in zip(preds1[:,2],preds2[:,2],preds3[:,2])]

# for soc
# stacker.fit(df_soc[['Predicted_svr', 'Predicted_ridge', 'Predicted_regressor']], labels[:,3])
# preds[:, 3] = stacker.predict(np.stack([preds3[:,3], preds2[:,3], preds1[:,3]], axis=-1)).astype(float)
preds[:,3] = [(i+j+k)/3 for i,j,k in zip(preds1[:,3],preds2[:,3],preds3[:,3])]

# for sand
# stacker.fit(df_sand[['Predicted_svr', 'Predicted_ridge', 'Predicted_regressor']], labels[:,4])
# preds[:, 4] = stacker.predict(np.stack([preds3[:,4], preds2[:,4], preds1[:,4]], axis=-1)).astype(float)
preds[:,4] = [(i+j+k)/3 for i,j,k in zip(preds1[:,4],preds2[:,4],preds3[:,4])]

sample = pd.read_csv('sample_submission.csv')
sample['Ca'] = preds[:,0]
sample['P'] = preds[:,1]
sample['pH'] = preds[:,2]
sample['SOC'] = preds[:,3]
sample['Sand'] = preds[:,4]

sample.to_csv('voting_regressor.csv', index = False)
# score = 0.45662